In [33]:
import pandas as pd
import numpy as np

In [10]:
reprisk_incidents = pd.read_csv('data/new_materiality_with_rri.csv') # Calculation in "CreateDataset" of "ESGmateriality" project
reprisk_incidents = reprisk_incidents[['gvkey', 'isin', 'cusip', 'reprisk_id', 'story_id', 'adopter', 'sic', 'SICS Codified Industry ', 'Codified SICS Sector ', 'severity', 'reach', 'novelty', 'incident_list', 'incident_date', 'cusip_cusip_tic', 'year', 'YearQuarter', 'materiality_reach', 'materiality_severity', 'materiality_car_1', 'materiality_car_5', 'materiality_rri', 'material_flag']]
reprisk_incidents

,gvkey,isin,cusip,reprisk_id,story_id,adopter,sic,SICS Codified Industry,Codified SICS Sector,severity,...,incident_date,cusip_cusip_tic,year,YearQuarter,materiality_reach,materiality_severity,materiality_car_1,materiality_car_5,materiality_rri,material_flag
0,213199.0,TW0002353000,NaN,10,826,1.0,3571.0,Hardware,Technology & Communications,2,...,2007-02-28,NaN,2007,2007Q1,0,0,0,0,0,1
1,213199.0,TW0002353000,NaN,10,1793,1.0,3571.0,Hardware,Technology & Communications,1,...,2007-09-09,NaN,2007,2007Q3,0,0,0,0,0,1
2,213199.0,TW0002353000,NaN,10,2335,1.0,3571.0,Hardware,Technology & Communications,1,...,2007-11-26,NaN,2007,2007Q4,1,0,0,0,0,0
3,213199.0,TW0002353000,NaN,10,2365,1.0,3571.0,Hardware,Technology & Communications,2,...,2007-08-21,NaN,2007,2007Q3,0,1,0,0,0,1
4,213199.0,TW0002353000,NaN,10,2513,1.0,3571.0,Hardware,Technology & Communications,1,...,2007-11-30,NaN,2007,2007Q4,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295249,295326.0,CNE100000TP3,NaN,99984,8294414,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,1,...,2022-08-30,NaN,2022,2022Q3,0,0,0,0,1,1
295250,295326.0,CNE100000TP3,NaN,99984,8339482,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,1,...,2023-01-01,NaN,2023,2023Q1,0,0,1,1,0,0
295251,295326.0,CNE100000TP3,NaN,99984,8362588,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,1,...,2022-10-28,NaN,2022,2022Q4,0,0,0,0,0,0
295252,295326.0,CNE100000TP3,NaN,99984,8376615,1.0,3678.0,Electrical & Electronic Equipment,Resource Transformation,2,...,2023-04-20,NaN,2023,2023Q2,1,0,0,0,0,1


In [22]:
reprisk_incidents.isna().sum()

gvkey                           0
isin                            0
cusip                      123533
reprisk_id                      0
story_id                        0
adopter                         0
sic                             0
SICS Codified Industry          0
Codified SICS Sector            0
severity                        0
reach                           0
novelty                         0
incident_list                   0
incident_date                   0
cusip_cusip_tic            123533
year                            0
YearQuarter                     0
materiality_reach               0
materiality_severity            0
materiality_car_1               0
materiality_car_5               0
materiality_rri                 0
material_flag                   0
dtype: int64

In [21]:
df = reprisk_incidents.copy()

# Ensure proper quarterly Periods
df['YearQuarter'] = pd.PeriodIndex(df['YearQuarter'].astype(str), freq='Q')

# Coerce flags to numeric and treat missing as 0
flag_cols = [
    'material_flag',
    'materiality_car_1', 'materiality_car_5',
    'materiality_rri', 'materiality_reach', 'materiality_severity'
]
df[flag_cols] = df[flag_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

key = ["gvkey", "YearQuarter"]

# 1) Aggregate incidents per firm-quarter
agg_fq = (
    df.groupby(key, dropna=False)
      .agg(
          n_material          = ('material_flag',         lambda s: (s == 1).sum()),
          n_nonmaterial       = ('material_flag',         lambda s: (s == 0).sum()),
          n_car_1_material    = ('materiality_car_1',     lambda s: (s == 1).sum()),
          n_car_5_material    = ('materiality_car_5',     lambda s: (s == 1).sum()),
          n_rri_material      = ('materiality_rri',       lambda s: (s == 1).sum()),
          n_reach_material    = ('materiality_reach',     lambda s: (s == 1).sum()),
          n_severity_material = ('materiality_severity',  lambda s: (s == 1).sum()),
      )
      .reset_index()
)

# 2) Build the BALANCED universe = all firms × all quarters (global min..max)
firms = (
    df['gvkey']
    .dropna()
    .unique()
)
qmin = df['YearQuarter'].min()
qmax = df['YearQuarter'].max()
quarters = pd.period_range(qmin, qmax, freq='Q')

balanced_idx = pd.MultiIndex.from_product([firms, quarters], names=key)
balanced_universe = pd.DataFrame(index=balanced_idx).reset_index()

# 3) Merge counts onto balanced universe and fill missing with zeros
reprisk = (
    balanced_universe
      .merge(agg_fq, on=key, how='left')
      .fillna(0)
      .sort_values(key)
      .reset_index(drop=True)
)

# 4) Cast counts to ints
count_cols = [
    'n_material','n_nonmaterial','n_car_1_material','n_car_5_material',
    'n_rri_material','n_reach_material','n_severity_material'
]
reprisk[count_cols] = reprisk[count_cols].astype('int16')

# (optional) If you prefer strings for YearQuarter:
reprisk['YearQuarter'] = reprisk['YearQuarter'].astype(str)

reprisk

,gvkey,YearQuarter,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material
0,1004.0,2007Q1,0,0,0,0,0,0,0
1,1004.0,2007Q2,0,0,0,0,0,0,0
2,1004.0,2007Q3,0,0,0,0,0,0,0
3,1004.0,2007Q4,0,0,0,0,0,0,0
4,1004.0,2008Q1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1091463,367430.0,2022Q4,0,0,0,0,0,0,0
1091464,367430.0,2023Q1,0,0,0,0,0,0,0
1091465,367430.0,2023Q2,0,0,0,0,0,0,0
1091466,367430.0,2023Q3,0,0,0,0,0,0,0


In [24]:
reprisk_merged = reprisk.merge(reprisk_incidents[['gvkey', 'cusip', 'SICS Codified Industry ', 'Codified SICS Sector ']].drop_duplicates(), on='gvkey', how='left')
reprisk_merged

,gvkey,YearQuarter,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector
0,1004.0,2007Q1,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
1,1004.0,2007Q2,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
2,1004.0,2007Q3,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
3,1004.0,2007Q4,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
4,1004.0,2008Q1,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
...,...,...,...,...,...,...,...,...,...,...,...,...
1091463,367430.0,2022Q4,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
1091464,367430.0,2023Q1,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
1091465,367430.0,2023Q2,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
1091466,367430.0,2023Q3,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage


In [28]:
df = reprisk_incidents.copy()

# --- Use existing year column ---
# Make sure it's integer-like
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')

key_y = ['gvkey', 'year']

# --- Incident counts per firm–year ---
counts = (
    df.groupby(key_y, dropna=False)
      .agg(
          n_material          = ('material_flag',        lambda s: (s == 1).sum()),
          n_nonmaterial       = ('material_flag',        lambda s: (s == 0).sum()),
          n_car_1_material    = ('materiality_car_1',    lambda s: (s == 1).sum()),
          n_car_5_material    = ('materiality_car_5',    lambda s: (s == 1).sum()),
          n_rri_material      = ('materiality_rri',      lambda s: (s == 1).sum()),
          n_reach_material    = ('materiality_reach',    lambda s: (s == 1).sum()),
          n_severity_material = ('materiality_severity', lambda s: (s == 1).sum()),
      )
      .reset_index()
)

# --- Balanced universe: all firms × all years (global min..max of df['year']) ---
firms = df['gvkey'].dropna().unique()
ymin  = int(df['year'].min())
ymax  = int(df['year'].max())
years = pd.Index(range(ymin, ymax + 1), name='year')

balanced = (
    pd.MultiIndex.from_product([firms, years], names=key_y)
      .to_frame(index=False)
)

# --- Merge counts; zeros where no incidents ---
reprisk_yearly = (
    balanced.merge(counts, on=key_y, how='left')
            .fillna(0)
            .sort_values(key_y)
            .reset_index(drop=True)
)

count_cols = [
    'n_material','n_nonmaterial','n_car_1_material','n_car_5_material',
    'n_rri_material','n_reach_material','n_severity_material'
]
reprisk_yearly[count_cols] = reprisk_yearly[count_cols].astype('int16')


reprisk_yearly


,gvkey,year,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material
0,1004.0,2007,0,0,0,0,0,0,0
1,1004.0,2008,0,0,0,0,0,0,0
2,1004.0,2009,0,0,0,0,0,0,0
3,1004.0,2010,0,1,0,0,0,1,0
4,1004.0,2011,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
272862,367430.0,2019,0,0,0,0,0,0,0
272863,367430.0,2020,0,0,0,0,0,0,0
272864,367430.0,2021,0,0,0,0,0,0,0
272865,367430.0,2022,0,0,0,0,0,0,0


In [29]:
reprisk_merged_yearly = reprisk_yearly.merge(reprisk_incidents[['gvkey', 'cusip', 'SICS Codified Industry ', 'Codified SICS Sector ']].drop_duplicates(), on='gvkey', how='left')
reprisk_merged_yearly

,gvkey,year,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector
0,1004.0,2007,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
1,1004.0,2008,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
2,1004.0,2009,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
3,1004.0,2010,0,1,0,0,0,1,0,00036110,Industrial Machinery & Goods,Resource Transformation
4,1004.0,2011,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation
...,...,...,...,...,...,...,...,...,...,...,...,...
272862,367430.0,2019,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
272863,367430.0,2020,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
272864,367430.0,2021,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage
272865,367430.0,2022,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage


In [17]:
reprisk.groupby('YearQuarter')['gvkey'].nunique()

YearQuarter
2007Q1    16051
2007Q2    16051
2007Q3    16051
2007Q4    16051
2008Q1    16051
          ...  
2022Q4    16051
2023Q1    16051
2023Q2    16051
2023Q3    16051
2023Q4    16051
Freq: Q-DEC, Name: gvkey, Length: 68, dtype: int64

# 1. Portfolios based on material incidents (yes/no)

In [31]:
reprisk_merged_yearly['n_car_1_material'].describe()

count    1.091468e+06
mean     2.785331e-02
std      2.758815e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.900000e+01
Name: n_car_1_material, dtype: float64

In [288]:
# df = your yearly panel
df = reprisk_merged_yearly.copy()
df.drop_duplicates(['cusip', 'year'], keep='last', inplace=True)
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')

metrics = {
    'material': 'n_material',
    'car_1':    'n_car_1_material',
    'car_5':    'n_car_5_material',
    'reach':    'n_reach_material',
    'severity': 'n_severity_material',
}

# --- Long memberships (one row per cusip-year-metric) ---
frames = []
for mname, mcol in metrics.items():
    tmp = df[['cusip', 'year', mcol]].copy()
    tmp['metric'] = mname
    tmp['portfolio'] = np.where(tmp[mcol] > 0, 'incident', 'no_incident')
    frames.append(tmp[['cusip', 'year', 'metric', 'portfolio']])

portfolios_long = pd.concat(frames, ignore_index=True)

# --- Wide memberships (one row per cusip-year; one column per metric) ---
wide = df[['cusip', 'year']].drop_duplicates().sort_values(['cusip', 'year']).copy()
wide_idx = wide.set_index(['cusip', 'year']).index

for mname, mcol in metrics.items():
    # make a Series aligned on (gvkey, year) index
    s = df.set_index(['cusip', 'year'])[mcol]
    s = (s > 0).map({True: 'incident', False: 'no_incident'})
    # align to the wide index
    wide[f'{mname}_portfolio'] = s.reindex(wide_idx).to_numpy()

# Example: counts per year & portfolio
portfolio_counts = (
    portfolios_long
    .groupby(['metric', 'year', 'portfolio'], as_index=False)
    .agg(n_firms=('cusip', 'nunique'))
    .sort_values(['metric', 'year', 'portfolio'])
)


In [289]:
def build_incident_flags(df):
    out = df.copy()

    # 1) find all portfolio label columns
    port_cols = [c for c in out.columns if c.endswith('_portfolio')]

    # 3) create 0/1 flags per portfolio column
    #    naming: noInc_<base>, inc_<base>  (e.g., noInc_material, inc_material)
    for c in port_cols:
        base = c.replace('_portfolio', '')  # e.g., 'material'
        out[f'noInc_{base}'] = (out[c] == 'no_incident').astype('int8')
        out[f'inc_{base}']   = (out[c] == 'incident').astype('int8')


    return out

In [290]:
# ---- use it ----
wide_flags = build_incident_flags(wide)  # replace df_inc with your DataFrame name

wide_flags


,cusip,year,material_portfolio,car_1_portfolio,car_5_portfolio,reach_portfolio,severity_portfolio,noInc_material,inc_material,noInc_car_1,inc_car_1,noInc_car_5,inc_car_5,noInc_reach,inc_reach,noInc_severity,inc_severity
81991,00030710,2007,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81992,00030710,2008,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81993,00030710,2009,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81994,00030710,2010,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
81995,00030710,2011,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272862,NaN,2019,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
272863,NaN,2020,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
272864,NaN,2021,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0
272865,NaN,2022,no_incident,no_incident,no_incident,no_incident,no_incident,1,0,1,0,1,0,1,0,1,0


# 2. Portfolios based on orthogonalization

In [90]:
import wrds
from pathlib import Path
from dotenv import load_dotenv
import os

project_root = Path(r"E:\GermanBusinessPanelTeam\Schrader\Forschung\ESGmateriality")

# ── Load WRDS creds from wrds.env in the project root ─────────────────────────
env_path = project_root / "wrds.env"
load_dotenv(dotenv_path=env_path)
# ── Connect to WRDS ───────────────────────────────────────────────────────────
db = wrds.Connection(
    wrds_username=os.getenv("WRDS_YALE_USERNAME"),
    wrds_password=os.getenv("WRDS_YALE_PASSWORD")
)

Loading library list...
Done


In [51]:
db.list_libraries()

['aha',
 'aha_common',
 'aha_hcris_recent',
 'aha_it_survey_recent',
 'aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'audit_oia',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'candid_samp',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'cddsamp',
 'ciq',
 'ciq_capstrct',
 'ciq_common',
 'ciq_keydev',
 'ciq_pplintel',
 'ciq_ratings',
 'ciq_transactions',
 'ciq_transcripts',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_filings',
 'comp_global_daily',
 'comp_na_daily_all',
 'comp_segments_hist_daily',
 'comp_snapshot',
 'compsamp',
 'com

In [162]:
db.list_tables(library='wrdssec')

['_qvards_',
 'admin',
 'admins',
 'adminstatecountry',
 'attachmentstab',
 'auditopinionseries',
 'authorizedparticipant',
 'authorizedparticipants',
 'bow_1993',
 'bow_1994',
 'bow_1995',
 'bow_1996',
 'bow_1997',
 'bow_1998',
 'bow_1999',
 'bow_2000',
 'bow_2001',
 'bow_2002',
 'bow_2003',
 'bow_2004',
 'bow_2005',
 'bow_2006',
 'bow_2007',
 'bow_2008',
 'bow_2009',
 'bow_2010',
 'bow_2011',
 'bow_2012',
 'bow_2013',
 'bow_2014',
 'bow_2015',
 'bow_2016',
 'bow_2017',
 'bow_2018',
 'bow_2019',
 'bow_2020',
 'bow_2021',
 'bow_2022',
 'bow_2023',
 'bow_2024',
 'bow_cossim',
 'bow_filingsummary',
 'bow_wordsummary',
 'broker',
 'brokerdealer',
 'brokerdealers',
 'brokerdealerstatecountry',
 'brokers',
 'brokerstatecountry',
 'ccoemployers',
 'ccostatecountry',
 'chars',
 'chiefcomplianceofficer',
 'chiefcomplianceofficers',
 'cisdf',
 'claimfiled',
 'classid',
 'closedendmanagementinvestment',
 'collateralmanager',
 'collateralmanagers',
 'commonstock',
 'commonstocks',
 'contact',
 'c

In [167]:
db.describe_table(library="wrdssec", table='wrds_13f_holdings')

Approximately 98307046 rows in wrdssec.wrds_13f_holdings.
['cik' 'coname' 'form' 'rdate' 'fdate' 'fname' 'nameofissuer'
 'titleofclass' 'cusip' 'value' 'sshprnamt' 'sshprnamttype' 'putcall'
 'investmentdiscretion' 'othermanager' 'sole' 'shared' 'none']


In [118]:
# ---------- tiny helper: keys + single Compustat pull ----------

def _keys(df):
    return (df[['gvkey','year']]
            .dropna()
            .assign(
                gvkey=lambda d: d['gvkey'].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(6),
                year =lambda d: d['year'].astype(int))
            .drop_duplicates())

def _pull_compustat(db, keys):
    """
    Pull minimal Compustat annual (INDL/STD/C) rows for calendar years needed:
      - for year t, we need calendar years in [t-1, t] (t for ROA, t-1 for others)
    Returns a DataFrame with columns used below + a 'calyear' (datadate.year).
    """
    y_min, y_max = int(keys['year'].min()), int(keys['year'].max())
    comp = db.raw_sql("""
        select gvkey, datadate,
               at, lt, ceq, seq, pstk, xrd, xad, sale, ib, mkvalt
        from comp.funda
        where indfmt='INDL' and datafmt='STD' and consol='C'
          and gvkey in %(g)s
          and extract(year from datadate) between %(a)s and %(b)s
    """, params={
        "g": tuple(keys['gvkey'].unique().tolist()),
        "a": y_min - 1,                      # need t-1 for most vars
        "b": y_max                           # need t for ROA
    }, date_cols=['datadate'])
    if comp.empty:
        comp = pd.DataFrame(columns=[
            'gvkey','datadate','at','lt','ceq','seq','pstk','xrd','xad','sale','ib','mkvalt'
        ])
    comp['calyear'] = pd.to_datetime(comp['datadate']).dt.year.astype('Int64')
    return comp

In [92]:
# ---------- 1) Size (Compustat mkvalt) ----------

def get_size(db, reprisk_merged_yearly):
    """
    Size = ln(mkvalt) using Compustat annual.
    For (gvkey, year=t), pick the *last* Compustat row with datadate in calendar year t-1.
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','size'])
    comp = _pull_compustat(db, keys)

    # map each row to target year t = calyear + 1; keep last within that calendar year
    sz = (comp.assign(year=lambda d: d['calyear'] + 1)
               .sort_values(['gvkey','year','datadate'])
               .drop_duplicates(['gvkey','year'], keep='last'))[['gvkey','year','mkvalt']]

    out = keys.merge(sz, on=['gvkey','year'], how='left')
    out['size'] = np.log(out['mkvalt'])  # mkvalt is typically in $ millions
    return out[['gvkey','year','size']]

In [93]:
keys_df = reprisk_merged_yearly[['gvkey', 'year']]
size_df = get_size(db, keys_df)
size_df.notna().sum()

gvkey    272867
year     272867
size      35218
dtype: int64

In [104]:
# ---------- 2) Book-to-Market (BTM) ----------

def get_btm(db, reprisk_merged_yearly):
    """
    BTM = Shareholders' Equity_{t-1} / mkvalt_{t-1}, with SE robustly:
      SE = seq → (ceq + pstk) → (at - lt)
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','btm'])
    comp = _pull_compustat(db, keys)

    # year-1 fundamentals → map to t
    fy1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                .sort_values(['gvkey','year','datadate'])
                .drop_duplicates(['gvkey','year'], keep='last')
                [['gvkey','year','seq','ceq','pstk','at','lt','mkvalt']])

    se = fy1['seq'].copy()
    se = se.fillna(fy1['ceq'] + fy1['pstk'])
    se = se.fillna(fy1['at'] - fy1['lt'])

    out = fy1[['gvkey','year','mkvalt']].copy()
    out['btm'] = se.values / out['mkvalt'].values
    return keys.merge(out[['gvkey','year','btm']], on=['gvkey','year'], how='left')

In [106]:
btm_df = get_btm(db, keys_df)
btm_df.notna().sum()

gvkey    272867
year     272867
btm       35217
dtype: int64

In [107]:
# ---------- 3) Leverage ----------

def get_leverage(db, reprisk_merged_yearly):
    """
    Leverage = lt / at using year-1 values (mapped to year t).
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','leverage'])
    comp = _pull_compustat(db, keys)

    fy1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                .sort_values(['gvkey','year','datadate'])
                .drop_duplicates(['gvkey','year'], keep='last')
                [['gvkey','year','at','lt','seq','ceq','pstk']])

    # if lt missing, fallback lt = at - SE
    se = fy1['seq'].fillna(fy1['ceq'] + fy1['pstk']).fillna(fy1['at'] - fy1['lt'])
    lt = fy1['lt'].fillna(fy1['at'] - se)

    out = fy1[['gvkey','year','at']].copy()
    out['leverage'] = lt.values / out['at'].values
    return keys.merge(out[['gvkey','year','leverage']], on=['gvkey','year'], how='left')

In [109]:
leverage_df = get_leverage(db, keys_df)
leverage_df.notna().sum()

gvkey       272867
year        272867
leverage     45447
dtype: int64

In [110]:
# ---------- 4) R&D intensity ----------

def get_rd_intensity(db, reprisk_merged_yearly):
    """
    R&D intensity = xrd / sale using year-1 values.
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','rd_intensity'])
    comp = _pull_compustat(db, keys)

    fy1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                .sort_values(['gvkey','year','datadate'])
                .drop_duplicates(['gvkey','year'], keep='last')
                [['gvkey','year','xrd','sale']])

    sale_pos = fy1['sale'].where(fy1['sale'] > 0, np.nan)
    out = fy1[['gvkey','year']].copy()
    out['rd_intensity'] = fy1['xrd'] / sale_pos
    return keys.merge(out, on=['gvkey','year'], how='left')

In [111]:
rd_df = get_rd_intensity(db, keys_df)
rd_df.notna().sum()

gvkey           272867
year            272867
rd_intensity     22934
dtype: int64

In [113]:
# ---------- 5) Advertising intensity ----------

def get_ad_intensity(db, reprisk_merged_yearly):
    """
    Advertising intensity = xad / sale using year-1 values.
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','ad_intensity'])
    comp = _pull_compustat(db, keys)

    fy1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                .sort_values(['gvkey','year','datadate'])
                .drop_duplicates(['gvkey','year'], keep='last')
                [['gvkey','year','xad','sale']])

    sale_pos = fy1['sale'].where(fy1['sale'] > 0, np.nan)
    out = fy1[['gvkey','year']].copy()
    out['ad_intensity'] = fy1['xad'] / sale_pos
    return keys.merge(out, on=['gvkey','year'], how='left')

In [114]:
ad_df = get_ad_intensity(db, keys_df)
ad_df.notna().sum()

gvkey           272867
year            272867
ad_intensity     19232
dtype: int64

In [116]:
# ---------- 6) ROA ----------

def get_roa(db, reprisk_merged_yearly):
    """
    ROA_t = IB_t / ((AT_t + AT_{t-1}) / 2)
      - IB_t, AT_t: Compustat row with datadate in calendar year t (last in that year)
      - AT_{t-1}:   from calendar year t-1 (last in that year), mapped to year t
    """
    keys = _keys(reprisk_merged_yearly)
    if keys.empty:
        return pd.DataFrame(columns=['gvkey','year','roa'])
    comp = _pull_compustat(db, keys)

    # current year t (ib, at)
    ft = (comp[comp['calyear'].isin(keys['year'].unique())]
              .assign(year=lambda d: d['calyear'])
              .sort_values(['gvkey','year','datadate'])
              .drop_duplicates(['gvkey','year'], keep='last')
              [['gvkey','year','ib','at']]
              .rename(columns={'ib':'ib_t','at':'at_t'}))

    # previous year (at_{t-1}) mapped to t
    ftm1 = (comp.assign(year=lambda d: d['calyear'] + 1)
                 .sort_values(['gvkey','year','datadate'])
                 .drop_duplicates(['gvkey','year'], keep='last')
                 [['gvkey','year','at']]
                 .rename(columns={'at':'at_tm1'}))

    base = keys.merge(ft, on=['gvkey','year'], how='left').merge(ftm1, on=['gvkey','year'], how='left')
    denom = (base['at_t'] + base['at_tm1']) / 2.0
    out = base[['gvkey','year']].copy()
    out['roa'] = base['ib_t'] / denom
    return out

In [117]:
roa_df = get_roa(db, keys_df)
roa_df.notna().sum()

gvkey    272867
year     272867
roa       43727
dtype: int64

In [169]:
# ---------- Institutional Ownership ----------

# Export unique cusips to txt
reprisk_merged_yearly['cusip'].dropna().astype(str).str.replace(r'\.0$', '', regex=True).drop_duplicates().to_csv('data/cusips.txt', index=False, header=False)


In [183]:
ownership_df = pd.read_csv('data/inst_ownership.csv')  # Exported from LSEG Stock Ownership Tool
ownership_df = ownership_df[['cusip', 'rdate', 'InstOwn_Perc']].sort_values(['cusip', 'rdate'])
ownership_df['year'] = pd.to_datetime(ownership_df['rdate']).dt.year - 1
ownership_df.drop_duplicates(['cusip', 'year'], keep='last', inplace=True)
ownership_df

,cusip,rdate,InstOwn_Perc,year
62148,00030710,2014-12-31,2.646183e-01,2013
70952,00030710,2015-12-31,4.817053e-01,2014
79872,00030710,2016-12-31,4.499213e-01,2015
88711,00030710,2017-12-31,6.102770e-01,2016
98929,00030710,2018-12-31,4.897175e-01,2017
...,...,...,...,...
104123,Y1477R20,2019-03-31,4.851384e-06,2018
114648,Y1477R20,2020-03-31,4.481560e-05,2019
146015,Y1477R20,2023-03-31,3.830040e-08,2022
112048,Y1501010,2019-12-31,4.962090e-05,2018


In [181]:
df.isna().sum()

gvkey                           0
year                            0
n_material                      0
n_nonmaterial                   0
n_car_1_material                0
n_car_5_material                0
n_rri_material                  0
n_reach_material                0
n_severity_material             0
cusip                      200736
SICS Codified Industry          0
Codified SICS Sector            0
size                       237649
btm                        237650
leverage                   227420
rd_intensity               249933
ad_intensity               253635
roa                        229140
rdate                      232309
InstOwn_Perc               232360
dtype: int64

In [197]:

outs = [
    size_df, btm_df, leverage_df, rd_df, ad_df, roa_df
]

df = reprisk_merged_yearly.copy()

for o in outs:
    o['year'] = pd.to_numeric(o['year'], errors='coerce').astype('Int64')
    o['gvkey'] = pd.to_numeric(o['gvkey'], errors='coerce').astype('Int64')
    df = df.merge(o, on=['gvkey','year'], how='left')

df =df.merge(ownership_df, left_on=['cusip','year'], right_on=['cusip','year'], how='left')
df

,gvkey,year,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector,size,btm,leverage,rd_intensity,ad_intensity,roa,rdate,InstOwn_Perc
0,1004.0,2007,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation,6.782904,0.47893,0.568136,<NA>,<NA>,0.058098,2008-12-31,1.140917
1,1004.0,2008,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation,7.111669,0.403071,0.537067,<NA>,<NA>,0.062351,2009-12-31,0.996335
2,1004.0,2009,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation,6.616792,0.782904,0.570301,<NA>,<NA>,0.058842,2010-12-31,0.990781
3,1004.0,2010,0,1,0,0,0,1,0,00036110,Industrial Machinery & Goods,Resource Transformation,6.34843,1.149232,0.523129,<NA>,<NA>,0.031007,2011-12-31,0.915529
4,1004.0,2011,0,0,0,0,0,0,0,00036110,Industrial Machinery & Goods,Resource Transformation,6.656514,0.960237,0.502779,<NA>,<NA>,0.045644,2012-12-31,0.841255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272862,367430.0,2019,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
272863,367430.0,2020,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
272864,367430.0,2021,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
272865,367430.0,2022,0,0,0,0,0,0,0,NaN,Agricultural Products,Food & Beverage,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [186]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [205]:
# configure
dep_vars  = ['n_car_1_material','n_car_5_material','n_rri_material',
             'n_reach_material','n_severity_material','n_material']
controls  = ['size','btm','leverage','rd_intensity','ad_intensity','roa','InstOwn_Perc']
industry_col = 'Codified SICS Sector '   # change if needed
year_col     = 'year'

def crosssec_residuals_year_indFE(df):
    out = df.copy()
    # make controls numeric once
    for c in controls:
        if c not in out.columns:
            raise KeyError(f"Missing control column: {c}")
        out[c] = pd.to_numeric(out[c], errors='coerce')
    if industry_col not in out.columns:
        raise KeyError(f"Missing industry column: {industry_col}")

    # init residual columns
    for y in dep_vars:
        out[f'resid_{y}'] = np.nan

    # loop dependent variables
    years = pd.to_numeric(out[year_col], errors='coerce')
    out[year_col] = years  # ensure numeric year

    for y in dep_vars:
        if y not in out.columns:
            continue
        out[y] = pd.to_numeric(out[y], errors='coerce')

        # run separate cross-section per year
        for yr, gidx in out.groupby(year_col).groups.items():
            if pd.isna(yr):
                continue
            sub = out.loc[gidx, [y] + controls + [industry_col]].copy()

            # drop all NA rows for this year's regression (as requested)
            sub = sub.replace([np.inf, -np.inf], np.nan).dropna(how='any')
            if sub.empty:
                continue

            # industry FE for this year's industries only
            ind_dum = pd.get_dummies(sub[industry_col], prefix='ind', drop_first=True, dtype=float)
            X = pd.concat([sub[controls].astype(float), ind_dum], axis=1)
            X = sm.add_constant(X)  # intercept
            yy = sub[y].astype(float)

            # need at least params+1 obs
            if X.shape[0] <= X.shape[1]:
                continue

            fit = sm.OLS(yy.to_numpy(), X.to_numpy()).fit()

            resid = pd.Series(yy.values - fit.fittedvalues, index=sub.index)
            out.loc[resid.index, f'resid_{y}'] = resid

    return out

In [206]:
# run
res_df = crosssec_residuals_year_indFE(df)

resid_n_car_1_material       0.972096
resid_n_car_5_material       0.972096
resid_n_rri_material         0.972096
resid_n_reach_material       0.972096
resid_n_severity_material    0.972096
resid_n_material             0.972096
dtype: float64


In [207]:
res_df.notna().sum()

gvkey                        272867
year                         272867
n_material                   272867
n_nonmaterial                272867
n_car_1_material             272867
n_car_5_material             272867
n_rri_material               272867
n_reach_material             272867
n_severity_material          272867
cusip                         72131
SICS Codified Industry       272867
Codified SICS Sector         272867
size                          35218
btm                           35217
leverage                      45447
rd_intensity                  22934
ad_intensity                  19232
roa                           43727
rdate                         38552
InstOwn_Perc                  38506
resid_n_car_1_material         7614
resid_n_car_5_material         7614
resid_n_rri_material           7614
resid_n_reach_material         7614
resid_n_severity_material      7614
resid_n_material               7614
dtype: int64

In [216]:
def add_industry_year_top_bottom_equal(
    df,
    resid_cols,
    industry_col='SICS Codified Industry ',
    year_col='year',
    tail='quintile',     # 'quintile' (20%) or 'decile' (10%)
    min_each=1           # at least this many per tail if the computed size is 0
):
    """
    Flags top/bottom *equal-sized* tails within each (industry, year) for each residual.
    'Top' = lowest residuals.
    Adds: topQ_<col>, botQ_<col>  OR topD_<col>, botD_<col> depending on 'tail'.
    """

    out = df.copy()
    if industry_col not in out.columns or year_col not in out.columns:
        raise KeyError("Missing industry/year column.")
    out[year_col] = pd.to_numeric(out[year_col], errors='coerce')

    group_keys = [industry_col, year_col]

    # share to select
    share = 0.20 if tail == 'quintile' else 0.10
    top_flag = 'topQ_' if tail == 'quintile' else 'topD_'
    bot_flag = 'botQ_' if tail == 'quintile' else 'botD_'

    for col in resid_cols:
        if col not in out.columns:
            continue
        out[col] = pd.to_numeric(out[col], errors='coerce')

        # init flags as 0; NaNs stay 0 by default (they won't be selected)
        out[top_flag + col] = np.int8(0)
        out[bot_flag + col] = np.int8(0)

        # work group by group to enforce exact counts
        for (ind, yr), idx in out.groupby(group_keys, dropna=False).groups.items():
            sub = out.loc[idx, [col]].copy()
            valid = sub[col].notna()
            if not valid.any():
                continue

            # deterministic ranks: ascending = better (top = lowest residuals)
            # method='first' breaks ties by row order, keeping counts exact
            ranks = sub.loc[valid, col].rank(method='first', ascending=True)
            n = int(valid.sum())

            k = max(min_each, int(np.floor(share * n)))  # equal count per tail
            if k == 0:  # belt-and-braces
                k = 1

            # indices of top k (smallest residuals) and bottom k (largest residuals)
            top_idx = ranks.nsmallest(k).index
            bot_idx = ranks.nlargest(k).index

            out.loc[top_idx, top_flag + col] = 1
            out.loc[bot_idx, bot_flag + col] = 1

    return out


In [221]:
resid_cols = [
    'resid_n_car_1_material','resid_n_car_5_material','resid_n_rri_material',
    'resid_n_reach_material','resid_n_severity_material','resid_n_material'
]

# Equal-sized quintile tails (20%)
df_ports_q = add_industry_year_top_bottom_equal(
    res_df, resid_cols, industry_col='Codified SICS Sector ', year_col='year',
    tail='quintile', min_each=1
)

# Equal-sized decile tails (10%)
df_ports_d = add_industry_year_top_bottom_equal(
    res_df, resid_cols, industry_col='Codified SICS Sector ', year_col='year',
    tail='decile', min_each=1
)

# sanity check: counts are equal (top vs bottom) per residual
for c in resid_cols:
    tq, bq = df_ports_q['topQ_'+c].sum(), df_ports_q['botQ_'+c].sum()
    td, bd = df_ports_d['topD_'+c].sum(), df_ports_d['botD_'+c].sum()
    print(c, 'Q:', tq, bq, '| D:', td, bd)

resid_n_car_1_material Q: 1469 1469 | D: 729 729
resid_n_car_5_material Q: 1469 1469 | D: 729 729
resid_n_rri_material Q: 1469 1469 | D: 729 729
resid_n_reach_material Q: 1469 1469 | D: 729 729
resid_n_severity_material Q: 1469 1469 | D: 729 729
resid_n_material Q: 1469 1469 | D: 729 729


In [270]:
df_ports_d

,gvkey,year,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_rri_material,n_reach_material,n_severity_material,cusip,...,topD_resid_n_car_5_material,botD_resid_n_car_5_material,topD_resid_n_rri_material,botD_resid_n_rri_material,topD_resid_n_reach_material,botD_resid_n_reach_material,topD_resid_n_severity_material,botD_resid_n_severity_material,topD_resid_n_material,botD_resid_n_material
0,1004.0,2007,0,0,0,0,0,0,0,00036110,...,0,0,0,0,0,0,0,0,0,0
1,1004.0,2008,0,0,0,0,0,0,0,00036110,...,0,0,0,0,0,0,0,0,0,0
2,1004.0,2009,0,0,0,0,0,0,0,00036110,...,0,0,0,0,0,0,0,0,0,0
3,1004.0,2010,0,1,0,0,0,1,0,00036110,...,0,0,0,0,0,0,0,0,0,0
4,1004.0,2011,0,0,0,0,0,0,0,00036110,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272862,367430.0,2019,0,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
272863,367430.0,2020,0,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
272864,367430.0,2021,0,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
272865,367430.0,2022,0,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0


# 3. Calculate Returns

In [223]:
project_root = Path(r"E:\GermanBusinessPanelTeam\Schrader\Forschung\ESGmateriality")

# ── Load WRDS creds from wrds.env in the project root ─────────────────────────
env_path = project_root / "wrds.env"
load_dotenv(dotenv_path=env_path)
# ── Connect to WRDS ───────────────────────────────────────────────────────────
db = wrds.Connection(
    wrds_username=os.getenv("WRDS_YALE_USERNAME"),
    wrds_password=os.getenv("WRDS_YALE_PASSWORD")
)

Loading library list...
Done


In [228]:
# ---------- PARAMETERS ----------
# set your window (CRSP coverage typical range is wide; adjust as needed)
START = "2007-01-01"
END   = "2024-12-31"
from pandas.tseries.offsets import MonthEnd

In [225]:

# ---------- 1) CRSP monthly returns (common stocks) ----------
# ret = total return; prc, shrout used for market equity; we’ll add delist return
sql_msf = f"""
select permno, date, ret, retx, prc, shrout
from crsp.msf
where date between '{START}' and '{END}'
"""
msf = db.raw_sql(sql_msf, date_cols=['date'])
msf['me'] = msf['prc'].abs() * msf['shrout'] * 1000.0

In [226]:
# ---- 2) Names table to get shrcd/exchcd as of date ----
# Use msenames (monthly security names history). Filter to primary share classes (optional).
sql_names = """
select permno, namedt, coalesce(nameendt, '9999-12-31') as nameendt, shrcd, exchcd
from crsp.msenames
"""
names = db.raw_sql(sql_names, date_cols=['namedt', 'nameendt'])

# Merge names onto msf by permno where date is within [namedt, nameendt]
msf = msf.merge(names, on='permno', how='left')
msf = msf[(msf['date'] >= msf['namedt']) & (msf['date'] <= msf['nameendt'])].copy()

# Optional: keep common shares on major exchanges
msf = msf[msf['shrcd'].isin([10, 11]) & msf['exchcd'].isin([1, 2, 3])]


In [227]:
msf

,permno,date,ret,retx,prc,shrout,me,namedt,nameendt,shrcd,exchcd
4,10001,2007-01-31,0.023279,0.023279,11.3584,2959.0,33609505.6,2004-12-27,2008-02-04,11,3
15,10002,2007-01-31,-0.158307,-0.158307,21.48,11166.0,239845680.0,2006-06-30,2013-02-15,11,3
17,10025,2007-01-31,-0.125305,-0.125305,46.63,7936.0,370055680.0,2004-06-10,2009-01-29,11,3
21,10026,2007-01-31,-0.002899,-0.002899,41.28,18541.0,765372480.0,2004-06-10,2019-08-04,11,3
37,10028,2007-01-31,0.078701,0.078701,2.7399,4913.0,13461128.7,2004-06-10,2007-10-30,11,3
...,...,...,...,...,...,...,...,...,...,...,...
8099800,93374,2024-12-31,-0.10221,-0.109907,62.44,103004.0,6431569760.0,2014-02-13,2024-12-31,11,1
8099832,93397,2024-12-31,-0.117446,-0.120144,24.46,18033.0,441087180.0,2019-09-11,2024-12-31,11,3
8099848,93426,2024-12-31,0.021768,0.021768,23.47,12216.0,286709520.0,2012-05-02,2024-12-31,11,1
8099862,93434,2024-12-31,0.133333,0.133333,7.99,2284.0,18249160.0,2024-10-18,2024-12-31,11,3


In [229]:
# ---- 3) Delisting returns & adjusted return ----
sql_dl = f"""
select permno, dlstdt as date, dlret
from crsp.msedelist
where dlstdt between '{START}' and '{END}'
"""
dl = db.raw_sql(sql_dl, date_cols=['date'])
dl['date'] = dl['date'] + MonthEnd(0)  # align to month-end

crsp = msf.merge(dl.rename(columns={'date':'dl_date'}),
                 left_on=['permno','date'], right_on=['permno','dl_date'], how='left')
crsp['dlret'] = pd.to_numeric(crsp['dlret'], errors='coerce')
crsp['ret']   = pd.to_numeric(crsp['ret'], errors='coerce')
crsp['ret_adj'] = (1.0 + crsp['ret'].fillna(0.0)) * (1.0 + crsp['dlret'].fillna(0.0)) - 1.0

In [230]:
# ---- 4) Lagged ME for VW weights (use ME at t-1 for month t) ----
crsp = crsp.sort_values(['permno','date'])
crsp['me_lag'] = crsp.groupby('permno')['me'].shift(1)

crsp_panel = crsp[['permno','date','ret_adj','me','me_lag','shrcd','exchcd']].copy()
crsp_panel

,permno,date,ret_adj,me,me_lag,shrcd,exchcd
0,10001,2007-01-31,0.023279,33609505.6,<NA>,11,3
4672,10001,2007-02-28,0.261621,42413910.0,33609505.6,11,3
9332,10001,2007-03-30,0.019732,43438940.0,42413910.0,11,3
13992,10001,2007-04-30,0.002764,43559020.0,43438940.0,11,3
18636,10001,2007-05-31,0.043418,45014970.0,43559020.0,11,3
...,...,...,...,...,...,...,...
832555,93436,2024-08-30,-0.077391,684004370400.000122,741380136746.400024,11,3
836414,93436,2024-09-30,0.221942,839047410000.0,684004370400.000122,11,3
840259,93436,2024-10-31,-0.045025,802033523100.599976,839047410000.0,11,3
844088,93436,2024-11-29,0.381469,1107984309600.000244,802033523100.599976,11,3


In [231]:
# ---- 5) Fama–French monthly factors ----
ff = db.raw_sql(f"""
select *
from ff.factors_monthly
where date between '{START}' and '{END}'
""", date_cols=['date'])

# Convert to decimals if table is in %
for col in ff.columns:
    if col.lower() != 'date' and pd.api.types.is_numeric_dtype(ff[col]):
        med = ff[col].abs().median(skipna=True)
        if pd.notna(med) and med > 0.5:  # crude but effective
            ff[col] = ff[col] / 100.0

ff = ff.rename(columns={
    'mktrf':'MKT_RF', 'smb':'SMB', 'hml':'HML', 'rf':'RF',
    'rmw':'RMW' if 'rmw' in ff.columns else 'RMW',
    'cma':'CMA' if 'cma' in ff.columns else 'CMA',
    'umd':'MOM' if 'umd' in ff.columns else 'MOM'
})
ff['date'] = ff['date'] + MonthEnd(0)

In [232]:
ff

,date,MKT_RF,SMB,HML,RF,year,month,MOM,dateff
0,2007-01-31,0.0138,0.0009,-0.0068,0.0044,20.07,0.01,0.0017,2007-01-31
1,2007-02-28,-0.0196,0.0123,-0.0008,0.0038,20.07,0.02,-0.0129,2007-02-28
2,2007-03-31,0.0071,0.0009,-0.0088,0.0043,20.07,0.03,0.0258,2007-03-30
3,2007-04-30,0.0349,-0.0221,-0.0144,0.0044,20.07,0.04,-0.0016,2007-04-30
4,2007-05-31,0.0323,0.0016,-0.0059,0.0041,20.07,0.05,-0.0033,2007-05-31
...,...,...,...,...,...,...,...,...,...
211,2024-08-31,0.016,-0.0349,-0.011,0.0048,20.24,0.08,0.0481,2024-08-30
212,2024-09-30,0.0172,-0.0013,-0.0277,0.004,20.24,0.09,-0.0062,2024-09-30
213,2024-10-31,-0.01,-0.0099,0.0086,0.0039,20.24,0.1,0.0298,2024-10-31
214,2024-11-30,0.0649,0.0446,0.0015,0.004,20.24,0.11,0.0099,2024-11-29


In [234]:
# --- pull CUSIP/NCUSIP name history ---
sql_names = """
select
    permno,
    namedt,
    coalesce(nameendt, '9999-12-31') as nameendt,
    cusip,
    ncusip
from crsp.msenames
"""
names = db.raw_sql(sql_names, date_cols=['namedt','nameendt'])

,permno,namedt,nameendt,cusip,ncusip
0,10000,1986-01-07,1986-12-03,68391610,68391610
1,10000,1986-12-04,1987-03-09,68391610,68391610
2,10000,1987-03-10,1987-06-11,68391610,68391610
3,10001,1986-01-09,1993-11-21,36720410,39040610
4,10001,1993-11-22,2004-06-09,36720410,29274A10
...,...,...,...,...,...
117825,93435,2010-06-14,2012-05-18,82936G20,82936G20
117826,93436,2010-06-29,2017-02-01,88160R10,88160R10
117827,93436,2017-02-02,2023-04-02,88160R10,88160R10
117828,93436,2023-04-03,2024-06-19,88160R10,88160R10


In [238]:
names.drop_duplicates(['permno'], keep='last', inplace=True)

,permno,namedt,nameendt,cusip,ncusip
2,10000,1987-03-10,1987-06-11,68391610,68391610
12,10001,2010-07-09,2017-08-03,36720410,36720410
18,10002,2006-06-30,2013-02-15,05978R10,05978R10
20,10003,1987-04-01,1995-12-15,39031810,39031810
21,10005,1986-01-15,1991-07-11,95815510,95815510
...,...,...,...,...,...
117819,93432,2010-06-08,2011-05-31,47737R10,47737R10
117822,93433,2013-04-24,2016-12-22,92870X30,92870X30
117824,93434,2024-10-18,2024-12-31,78513530,78513530
117825,93435,2010-06-14,2012-05-18,82936G20,82936G20


In [239]:
# --- align to month-end dates in crsp_panel ---
# merge then filter rows where crsp date is within [namedt, nameendt]
crsp_with_cusip = crsp_panel.merge(names, on='permno', how='left')
crsp_with_cusip

,permno,date,ret_adj,me,me_lag,shrcd,exchcd,namedt,nameendt,cusip,ncusip
0,10001,2007-01-31,0.023279,33609505.6,<NA>,11,3,2010-07-09,2017-08-03,36720410,36720410
1,10001,2007-02-28,0.261621,42413910.0,33609505.6,11,3,2010-07-09,2017-08-03,36720410,36720410
2,10001,2007-03-30,0.019732,43438940.0,42413910.0,11,3,2010-07-09,2017-08-03,36720410,36720410
3,10001,2007-04-30,0.002764,43559020.0,43438940.0,11,3,2010-07-09,2017-08-03,36720410,36720410
4,10001,2007-05-31,0.043418,45014970.0,43559020.0,11,3,2010-07-09,2017-08-03,36720410,36720410
...,...,...,...,...,...,...,...,...,...,...,...
847888,93436,2024-08-30,-0.077391,684004370400.000122,741380136746.400024,11,3,2024-06-20,2024-12-31,88160R10,88160R10
847889,93436,2024-09-30,0.221942,839047410000.0,684004370400.000122,11,3,2024-06-20,2024-12-31,88160R10,88160R10
847890,93436,2024-10-31,-0.045025,802033523100.599976,839047410000.0,11,3,2024-06-20,2024-12-31,88160R10,88160R10
847891,93436,2024-11-29,0.381469,1107984309600.000244,802033523100.599976,11,3,2024-06-20,2024-12-31,88160R10,88160R10


# 4. Portfolio Returns

In [240]:
# ----------------------------
# 0) Helpers
# ----------------------------

def choose_primary_class(df, by=['cusip','date']):
    """
    If multiple PERMNOs share the same cusip8 at a date,
    keep the one with the largest lagged market cap (me_lag).
    """
    df = df.copy()
    df['_rank'] = df.groupby(by)['me_lag'].rank(method='first', ascending=False)
    out = df.loc[df['_rank'] == 1].drop(columns=['_rank'])
    return out

In [242]:
# ----------------------------
# 1) Prepare CRSP monthly panel
# ----------------------------
crsp = crsp_with_cusip.copy()
crsp['date'] = pd.to_datetime(crsp['date'])


# If same cusip8 maps to multiple permnos per month, keep largest me_lag
crsp_1class = choose_primary_class(crsp, by=['cusip','date'])

In [ ]:
# ----------------------------
# 2) Prepare factors
# ----------------------------
ff = ff.copy()
ff['date'] = pd.to_datetime(ff['date']) + MonthEnd(0)

In [295]:
# ----------------------------
# 3) Build holdings months from yearly portfolio file
# ----------------------------
ports = wide_flags.copy()
ports['year'] = pd.to_numeric(ports['year'], errors='coerce').astype('Int64')

# --- detect ALL portfolio-flag columns automatically ---
flag_cols = [c for c in ports.columns
             if (c.startswith(('topQ_', 'botQ_', 'topD_', 'botD_', 'noInc_', 'inc_')))]

if not flag_cols:
    raise ValueError("No portfolio flags found. Expecting columns like 'topQ_resid_...' etc.")

# --- expand each firm-year row to Jan–Dec of (year+1) and carry ALL flags ---
def _expand_hold_months(row):
    y = int(row['year']) + 1
    months = pd.date_range(f'{y}-01-01', f'{y}-12-31', freq='ME')  # month-ends
    out = pd.DataFrame({'date': months})
    out['cusip'] = row['cusip']
    out['hold_from_year'] = int(row['year'])
    # copy all detected flags into the monthly rows
    for col in flag_cols:
        out[col] = row.get(col, 0)
    return out

hold_list = [_expand_hold_months(r) for _, r in ports.iterrows()]
holds_all = pd.concat(hold_list, ignore_index=True)

# keep only months where a stock is in at least one portfolio (any flag == 1)
any_flag = holds_all[flag_cols].fillna(0).astype('int8').sum(axis=1) > 0
holds_all = holds_all.loc[any_flag].reset_index(drop=True)

In [296]:
holds_all

,date,cusip,hold_from_year,noInc_material,inc_material,noInc_car_1,inc_car_1,noInc_car_5,inc_car_5,noInc_reach,inc_reach,noInc_severity,inc_severity
0,2008-01-31,00030710,2007,1,0,1,0,1,0,1,0,1,0
1,2008-02-29,00030710,2007,1,0,1,0,1,0,1,0,1,0
2,2008-03-31,00030710,2007,1,0,1,0,1,0,1,0,1,0
3,2008-04-30,00030710,2007,1,0,1,0,1,0,1,0,1,0
4,2008-05-31,00030710,2007,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
865771,2024-08-31,NaN,2023,1,0,1,0,1,0,1,0,1,0
865772,2024-09-30,NaN,2023,1,0,1,0,1,0,1,0,1,0
865773,2024-10-31,NaN,2023,1,0,1,0,1,0,1,0,1,0
865774,2024-11-30,NaN,2023,1,0,1,0,1,0,1,0,1,0


In [297]:
# ----------------------------
# 4) Join holdings to CRSP returns by cusip8 + date
# ----------------------------
panel = holds_all.merge(crsp_1class[['cusip','date','permno','ret_adj','me','me_lag']],
                    on=['cusip','date'], how='left')

# If some holdings are missing returns (e.g., no CRSP coverage that month), drop them
panel = panel.dropna(subset=['ret_adj'])
panel

,date,cusip,hold_from_year,noInc_material,inc_material,noInc_car_1,inc_car_1,noInc_car_5,inc_car_5,noInc_reach,inc_reach,noInc_severity,inc_severity,permno,ret_adj,me,me_lag
83,2014-12-31,00030710,2013,1,0,1,0,1,0,1,0,1,0,14945,0.071379,651824520.0,608397660.0
86,2015-03-31,00030710,2014,1,0,1,0,1,0,1,0,1,0,14945,-0.15032,666246460.0,758705190.0
87,2015-04-30,00030710,2014,1,0,1,0,1,0,1,0,1,0,14945,0.133421,754929460.0,666246460.0
89,2015-06-30,00030710,2014,1,0,1,0,1,0,1,0,1,0,14945,0.120947,951350400.0,848702400.0
90,2015-07-31,00030710,2014,1,0,1,0,1,0,1,0,1,0,14945,-0.127411,830138400.0,951350400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851948,2011-09-30,98992210,2010,1,0,1,0,1,0,1,0,1,0,77787,-0.155327,835827780.0,989215200.0
851949,2011-10-31,98992210,2010,1,0,1,0,1,0,1,0,1,0,77787,0.001855,837378070.0,835827780.0
851950,2011-11-30,98992210,2010,1,0,1,0,1,0,1,0,1,0,77787,0.217403,1019426410.0,837378070.0
851952,2012-01-31,98992210,2011,1,0,1,0,1,0,1,0,1,0,77787,0.085945,1523485050.0,1393371720.0


In [298]:
# create panel with firsm financials, portfolio flags, returns, lagged ME
panel_firms = panel.merge(df_ports_q[['cusip','year','Codified SICS Sector ', 'SICS Codified Industry ', 'size', 'btm', 'leverage', 'ad_intensity', 'rd_intensity', 'roa', 'InstOwn_Perc']],
                          left_on=['cusip','hold_from_year'], right_on=['cusip','year'], how='left')
panel_firms

,date,cusip,hold_from_year,noInc_material,inc_material,noInc_car_1,inc_car_1,noInc_car_5,inc_car_5,noInc_reach,...,year,Codified SICS Sector,SICS Codified Industry,size,btm,leverage,ad_intensity,rd_intensity,roa,InstOwn_Perc
0,2014-12-31,00030710,2013,1,0,1,0,1,0,1,...,2013,Health Care,Health Care Delivery,-5.381699,408.913043,0.696052,<NA>,0.0,0.011624,0.264618
1,2015-03-31,00030710,2014,1,0,1,0,1,0,1,...,2014,Health Care,Health Care Delivery,1.568616,1.704792,0.709388,<NA>,0.0,0.06633,0.481705
2,2015-04-30,00030710,2014,1,0,1,0,1,0,1,...,2014,Health Care,Health Care Delivery,1.568616,1.704792,0.709388,<NA>,0.0,0.06633,0.481705
3,2015-06-30,00030710,2014,1,0,1,0,1,0,1,...,2014,Health Care,Health Care Delivery,1.568616,1.704792,0.709388,<NA>,0.0,0.06633,0.481705
4,2015-07-31,00030710,2014,1,0,1,0,1,0,1,...,2014,Health Care,Health Care Delivery,1.568616,1.704792,0.709388,<NA>,0.0,0.06633,0.481705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266069,2011-09-30,98992210,2010,1,0,1,0,1,0,1,...,2010,Health Care,Medical Equipment & Supplies,6.116453,0.618986,0.245502,0.005779,0.102481,0.047143,0.977690
266070,2011-10-31,98992210,2010,1,0,1,0,1,0,1,...,2010,Health Care,Medical Equipment & Supplies,6.116453,0.618986,0.245502,0.005779,0.102481,0.047143,0.977690
266071,2011-11-30,98992210,2010,1,0,1,0,1,0,1,...,2010,Health Care,Medical Equipment & Supplies,6.116453,0.618986,0.245502,0.005779,0.102481,0.047143,0.977690
266072,2012-01-31,98992210,2011,1,0,1,0,1,0,1,...,2011,Health Care,Medical Equipment & Supplies,6.542377,0.451884,0.272052,0.005117,0.103451,0.069125,0.903878


In [299]:
panel_firms.to_csv('Output/panel_firms_binary.csv', index=False)

In [301]:
def compute_all_quintile_portfolio_returns(panel):
    panel = panel.copy()
    panel['date'] = pd.to_datetime(panel['date'])

    # 1) detect all top/bottom quintile flags
    top_flags = [c for c in panel.columns if c.startswith('noInc_')]
    bot_flags = [c for c in panel.columns if c.startswith('inc_')]

    if not top_flags or not bot_flags:
        raise ValueError("No topQ_/botQ_ columns found in panel.")

    # 2) helper: return series for one flag (EW or VW)
    def _flag_returns(df, flag_col, weight='EW'):
        p = df.loc[df[flag_col] == 1].copy()
        if p.empty:
            return pd.Series(dtype=float, name=f"{flag_col}_{weight}")
        if weight == 'EW':
            return (p.groupby('date')['ret_adj']
                     .mean()
                     .rename(f"{flag_col}_{weight}"))
        # VW: lagged ME as start-of-month weight
        p = p.dropna(subset=['me_lag']).copy()
        if p.empty:
            return pd.Series(dtype=float, name=f"{flag_col}_{weight}")
        wsum = p.groupby('date')['me_lag'].sum().rename('W')
        p = p.merge(wsum, on='date', how='left')
        p['w'] = np.where(p['W'] > 0, p['me_lag'] / p['W'], np.nan)
        return (p['w'] * p['ret_adj']).groupby(p['date']).sum().rename(f"{flag_col}_{weight}")

    # 3) compute EW/VW for every individual flag
    parts = []
    for flag in sorted(set(top_flags + bot_flags)):
        parts.append(_flag_returns(panel, flag, 'EW'))
        parts.append(_flag_returns(panel, flag, 'VW'))

    ts_flags = pd.concat(parts, axis=1).sort_index()

    # 4) build long–short per “base signal”: topQ_xxx − botQ_xxx (both EW and VW)
    #    infer base names by stripping the prefixes
    def _basename(col):  # 'topQ_resid_n_material' -> 'resid_n_material'
        return col.split('noInc_', 1)[1] if col.startswith('noInc_') else col.split('inc_', 1)[1]

    top_map = {_basename(c): c for c in top_flags}
    bot_map = {_basename(c): c for c in bot_flags}
    common_bases = sorted(set(top_map).intersection(bot_map))

    for base in common_bases:
        tflag, bflag = top_map[base], bot_map[base]
        # EW spread
        t_ew = f"{tflag}_EW"
        b_ew = f"{bflag}_EW"
        if t_ew in ts_flags.columns and b_ew in ts_flags.columns:
            ts_flags[f"LS_EW_{base}"] = ts_flags[t_ew] - ts_flags[b_ew]
        # VW spread
        t_vw = f"{tflag}_VW"
        b_vw = f"{bflag}_VW"
        if t_vw in ts_flags.columns and b_vw in ts_flags.columns:
            ts_flags[f"LS_VW_{base}"] = ts_flags[t_vw] - ts_flags[b_vw]

    return ts_flags

# ---- run it ----
ts_all = compute_all_quintile_portfolio_returns(panel)
ts_all

,inc_car_1_EW,inc_car_1_VW,inc_car_5_EW,inc_car_5_VW,inc_material_EW,inc_material_VW,inc_reach_EW,inc_reach_VW,inc_severity_EW,inc_severity_VW,...,LS_EW_car_1,LS_VW_car_1,LS_EW_car_5,LS_VW_car_5,LS_EW_material,LS_VW_material,LS_EW_reach,LS_VW_reach,LS_EW_severity,LS_VW_severity
date,,,,,,,,,,,,,,,,,,,,,
2008-01-31,-0.043552,-0.059848,-0.037122,-0.050044,-0.052275,-0.066274,-0.046602,-0.062281,-0.023825,-0.065291,...,-0.008517,-0.008046,-0.015301,-0.022452,0.000592,0.001747,-0.005321,-0.00434,-0.028757,-0.000146
2008-02-29,-0.017988,-0.028959,-0.016238,-0.03099,-0.013421,-0.026427,-0.024984,-0.03348,-0.030951,-0.023468,...,-0.009425,0.003093,-0.011315,0.00613,-0.01501,-0.00081,-0.002136,0.009246,0.004024,-0.004252
2008-03-31,-0.013511,0.005017,-0.01288,0.004214,-0.004687,0.006752,-0.009747,0.012905,-0.022878,0.003264,...,0.00268,-0.010228,0.002036,-0.00919,-0.006885,-0.017683,-0.001231,-0.02071,0.012305,-0.006719
2008-04-30,0.064591,0.042797,0.061127,0.039771,0.059162,0.043015,0.069173,0.040434,0.041424,0.053966,...,-0.026485,0.004769,-0.023016,0.009299,-0.022053,0.006211,-0.031101,0.007845,-0.002329,-0.009986
2008-06-30,-0.079437,-0.062494,-0.080191,-0.064553,-0.077088,-0.074078,-0.070016,-0.069754,-0.064777,-0.049494,...,-0.019852,-0.026206,-0.019183,-0.023521,-0.023603,-0.013261,-0.029527,-0.015092,-0.034765,-0.039463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,0.038093,0.073672,0.048165,0.068224,0.044393,0.05248,0.027095,0.047713,0.026821,0.048931,...,0.019596,-0.047401,0.008636,-0.039366,0.01927,-0.018808,0.031028,0.001989,0.03099,-0.000127
2024-07-31,0.041408,-0.002599,0.052251,0.002575,0.067843,0.008925,0.041667,0.007048,0.044218,0.001449,...,0.040366,0.03539,0.028673,0.026702,0.017408,0.033498,0.03932,0.014377,0.036192,0.020885
2024-09-30,0.030945,0.026545,0.032015,0.021478,0.01889,0.023167,0.027947,0.034433,0.020034,0.025705,...,-0.014236,-0.006164,-0.015517,0.003997,-0.001608,0.00123,-0.010754,-0.019371,-0.002238,-0.003463


In [302]:
def add_excess_returns_for_all(ts_all, ff, rf_col='RF', subtract_rf_ls=False):
    """
    Merge RF and create _ex (excess) versions of all portfolio series.
    - Subtract RF from all *legs* (top/bot, EW/VW).
    - For LS spreads, subtraction is optional (subtract_rf_ls flag).
    Returns a new DataFrame with added *_ex columns.
    """
    ts = ts_all.merge(ff[['date', rf_col]], on='date', how='left').copy()

    # Identify columns
    leg_cols = [c for c in ts.columns
                if (c.endswith('_EW') or c.endswith('_VW'))
                and not c.startswith('LS_')]  # top/bot legs only

    ls_cols = [c for c in ts.columns
               if c.startswith('LS_EW_') or c.startswith('LS_VW_') or c.startswith('LS_')]

    # Excess for legs (always subtract RF)
    for col in leg_cols:
        ts[f"{col}_ex"] = ts[col] - ts[rf_col]

    # Excess for LS (optional)
    if subtract_rf_ls:
        for col in ls_cols:
            ts[f"{col}_ex"] = ts[col] - ts[rf_col]

    return ts

# ---- example usage ----
# ts_all: from compute_all_quintile_portfolio_returns(panel)
# ff: your factor DataFrame with 'date' and 'RF' (in decimals)
# Option A (common): don't subtract RF from LS
ts_with_ex = add_excess_returns_for_all(ts_all, ff, rf_col='RF', subtract_rf_ls=False)

In [303]:
ts_with_ex.to_csv('Output/portfolio_returns_binary.csv', index=False)

In [249]:
# Choose factor set available in your ff file:
factor_cols = [c for c in ['MKT_RF','SMB','HML','RMW','CMA','MOM'] if c in ts_with_ex.columns]

def span_reg(ycol, df, factors=factor_cols, nw_lags=6):
    d = df.dropna(subset=[ycol] + factors).copy()
    y = d[ycol].astype(float)
    X = sm.add_constant(d[factors].astype(float))
    fit = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': nw_lags})
    alpha = fit.params['const']
    t_alpha = fit.tvalues['const']
    return fit, alpha, t_alpha

In [250]:
# Example: alpha for long–short (EW and VW)
fit_EW, alpha_EW, t_EW = span_reg('LS_EW_ex', ts)
fit_VW, alpha_VW, t_VW = span_reg('LS_VW_ex', ts)

print("Long–Short (EW): alpha = {:.4%}, t = {:.2f}".format(alpha_EW, t_EW))
print("Long–Short (VW): alpha = {:.4%}, t = {:.2f}".format(alpha_VW, t_VW))

Long–Short (EW): alpha = -0.4482%, t = -2.04
Long–Short (VW): alpha = -0.1671%, t = -1.00


In [251]:
# You can also get alphas for the long and short legs separately:
fit_long_EW, a_l_ew, t_l_ew = span_reg(f'{LONG_FLAG}_EW_ex', ts)
fit_short_EW, a_s_ew, t_s_ew = span_reg(f'{SHORT_FLAG}_EW_ex', ts)
print("Long-only (EW):  alpha = {:.4%}, t = {:.2f}".format(a_l_ew, t_l_ew))
print("Short-only (EW): alpha = {:.4%}, t = {:.2f}".format(a_s_ew, t_s_ew))


Long-only (EW):  alpha = 0.3574%, t = 2.14
Short-only (EW): alpha = 0.7308%, t = 3.69
